# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770416984711                   -0.52    9.0    188ms
  2   -2.771692091418       -2.89       -1.32    1.0    130ms
  3   -2.771714336240       -4.65       -2.45    1.0    108ms
  4   -2.771714633283       -6.53       -3.14    1.0    141ms
  5   -2.771714714203       -7.09       -3.99    2.0    108ms
  6   -2.771714715147       -9.03       -4.68    1.0   94.8ms
  7   -2.771714715249       -9.99       -5.93    2.0    128ms
  8   -2.771714715249      -12.67       -6.07    2.0    112ms
  9   -2.771714715250      -12.85       -6.89    1.0    112ms
 10   -2.771714715250      -13.86       -7.71    2.0    123ms
 11   -2.771714715250      -14.12       -8.81    2.0    120ms


-0.00013457360524221054

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770474691919                   -0.53    9.0    369ms
  2   -2.771766697329       -2.89       -1.31    1.0    176ms
  3   -2.771800947978       -4.47       -2.63    1.0   91.7ms
  4   -2.771802072956       -5.95       -4.11    2.0    140ms
  5   -2.771802074424       -8.83       -5.18    1.0    117ms
  6   -2.771802074476      -10.28       -5.65    2.0    108ms
  7   -2.771802074476      -12.33       -6.19    1.0    120ms
  8   -2.771802074476      -13.62       -7.13    1.0   98.3ms
  9   -2.771802074476      -14.18       -7.35    2.0    131ms
 10   -2.771802074476   +  -14.10       -8.01    1.0    107ms


0.01761222152540701

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457360524221054
Displaced dipole:  0.01761222152540701
Polarizability :   1.774679513064922


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920783599e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551498629e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852767923492e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694603522305e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088786413993e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.275252207663e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.278760873300e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.278760873300e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.948526903466e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.135482953648e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.166331063675e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.166331063675e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.166266955614639e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.